In [1]:
!pip install -q sequence
!pip install -q matplotlib
!pip install -q ipywidgets

In [2]:
def NewNetworkManager(owner: "QuantumRouter") -> "NetworkManager":
    manager = NetworkManager(owner, [])
    routing = StaticRoutingProtocol(owner, owner.name + ".StaticRoutingProtocol", {})
    rsvp = ResourceReservationProtocol(owner, owner.name + ".RSVP")
    routing.upper_protocols.append(rsvp)
    rsvp.lower_protocols.append(routing)
    manager.load_stack([routing, rsvp])
    return manager

In [3]:
from sequence.topology.router_net_topo import RouterNetTopo


network_config = "2.json"
network_topo = RouterNetTopo(network_config)
tl = network_topo.get_timeline()

/usr/local/lib/python3.10/dist-packages/qutip/__init__.py:66: UserWarning: The new version of Cython, (>= 3.0.0) is not supported.
  warnings.warn(


In [4]:
def set_parameters(topology: RouterNetTopo):
    # set memory parameters
    MEMO_FREQ = 2e3
    MEMO_EXPIRE = 0
    MEMO_EFFICIENCY = 1
    MEMO_FIDELITY = 0.9049367588934053
    for node in topology.get_nodes_by_type(RouterNetTopo.QUANTUM_ROUTER):
        memory_array = node.get_components_by_type("MemoryArray")[0]
        memory_array.update_memory_params("frequency", MEMO_FREQ)
        memory_array.update_memory_params("coherence_time", MEMO_EXPIRE)
        memory_array.update_memory_params("efficiency", MEMO_EFFICIENCY)
        memory_array.update_memory_params("raw_fidelity", MEMO_FIDELITY)

    # set detector parameters
    DETECTOR_EFFICIENCY = 0.9
    DETECTOR_COUNT_RATE = 5e7
    DETECTOR_RESOLUTION = 100
    for node in topology.get_nodes_by_type(RouterNetTopo.BSM_NODE):
        bsm = node.get_components_by_type("SingleAtomBSM")[0]
        bsm.update_detectors_params("efficiency", DETECTOR_EFFICIENCY)
        bsm.update_detectors_params("count_rate", DETECTOR_COUNT_RATE)
        bsm.update_detectors_params("time_resolution", DETECTOR_RESOLUTION)
    # set entanglement swapping parameters
    SWAP_SUCC_PROB = 0.7
    SWAP_DEGRADATION = 0.99
    for node in topology.get_nodes_by_type(RouterNetTopo.QUANTUM_ROUTER):
        node.network_manager.protocol_stack[1].set_swapping_success_rate(SWAP_SUCC_PROB)
        node.network_manager.protocol_stack[1].set_swapping_degradation(SWAP_DEGRADATION)

    # set quantum channel parameters
    ATTENUATION = 1e-5
    QC_FREQ = 1e11
    for qc in topology.get_qchannels():
        qc.attenuation = ATTENUATION
        qc.frequency = QC_FREQ

In [5]:
set_parameters(network_topo)


In [6]:
# the start and end nodes may be edited as desired
start_node_name = "end1"
end_node_name = "end2"
node1 = node2 = None

for router in network_topo.get_nodes_by_type(RouterNetTopo.QUANTUM_ROUTER):
    if router.name == start_node_name:
        node1 = router
    elif router.name == end_node_name:
        node2 = router

nm = node1.network_manager
nm.request(end_node_name, start_time=1e12, end_time=10e12, memory_size=10, target_fidelity=0.9)

tl.init()
tl.run()

print(node1, "memories")
print("Index:\tEntangled Node:\tFidelity:\tEntanglement Time:")
for info in node1.resource_manager.memory_manager:
		print("{:6}\t{:15}\t{:9}\t{}".format(str(info.index),
                                         str(info.remote_node),
                                         str(info.fidelity),
                                         str(info.entangle_time * 1e-12)))


end1 memories
Index:	Entangled Node:	Fidelity:	Entanglement Time:
0     	end2           	0.912173610973409	1.37393125147
1     	end2           	0.912173610973409	1.99534750287
2     	end2           	0.912173610973409	1.67300125205
3     	end2           	0.912173610973409	1.30040375128
4     	None           	0        	-1e-12
5     	None           	0        	-1e-12
6     	end2           	0.912173610973409	1.10304750046
7     	end2           	0.912173610973409	1.32565750128
8     	center         	0.9049367588934053	1.99885000289
9     	end2           	0.912173610973409	1.05478125025
10    	None           	0        	-1e-12
11    	None           	0        	-1e-12
12    	None           	0        	-1e-12
13    	None           	0        	-1e-12
14    	None           	0        	-1e-12
15    	None           	0        	-1e-12
16    	None           	0        	-1e-12
17    	None           	0        	-1e-12
18    	None           	0        	-1e-12
19    	None           	0        	-1e-12
20    	None  